# https://lovit.github.io/nlp/2018/08/28/levenshtein_hangle/
   
      
### s1, s2에 대해 list의 각 element를 순서대로 확인   
   
Python str은 list of characters이기 떄문에 enumrate(str)은 한 글자씩 yield함

In [6]:
def levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug)
    
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        
        if debug:
            print(current_row[1:])
        
        previous_row = current_row
        
    return previous_row[-1]

In [20]:
s1 = '꿈을꾸는아이'
s2 = '아이오아이'
levenshtein(s1, s2, debug=True)
    
print('\n')
    
s3 = '감나 다라'
s4 = ' 가나다라'
levenshtein(s3, s4, debug=True)
    
print('\n')
    
s5 = '꿈을 꾸는 아이'
s6 = '아이는 꿈을 꿔요'
levenshtein(s5, s6, debug=True)

[1, 2, 3, 4, 5]
[2, 2, 3, 4, 5]
[3, 3, 3, 4, 5]
[4, 4, 4, 4, 5]
[4, 5, 5, 4, 5]
[5, 4, 5, 5, 4]


[1, 2, 3, 4, 5]
[2, 2, 2, 3, 4]
[2, 3, 3, 3, 4]
[3, 3, 4, 3, 4]
[4, 4, 4, 4, 3]


[1, 2, 3, 4, 5, 6, 6, 7]
[2, 2, 3, 4, 5, 6, 7, 6]
[3, 3, 3, 4, 4, 5, 6, 7]
[4, 4, 3, 4, 5, 4, 5, 6]
[4, 5, 4, 4, 5, 5, 5, 6]
[5, 4, 5, 5, 5, 6, 6, 6]
[6, 5, 4, 5, 6, 5, 6, 7]
[7, 6, 5, 5, 6, 6, 6, 7]
[8, 7, 6, 6, 6, 7, 7, 7]


7

### 띄어쓰기 기준으로 나뉘어지는 어절로 거리 계산   
   
split()해서 list of str이 되도록 입력라면 어절 단위로 거리가 계산

In [8]:
s7 = '꿈을 꾸는 아이'
s8 = '아이는 꿈을 꿔요'
levenshtein(s7.split(), s8.split(), debug=True)

[1, 1, 2]
[2, 2, 2]
[3, 3, 3]


3

### cost 계산 (c1이 c2로 변하는 비용)   
   
특정한 글자가 서로 자주 교차될 때 (ex_ 서비스 -> 써비스)   
초성이 된소리가 되거나, 종성에 'ㅇ' 받침이 추가되는 경우가 잦다면   
이 때의 edit distance의 비용을 글자에 따라 다르게 부과

In [9]:
def levenshtein2(s1, s2, cost=None, debug=False):
    if len(s1) < len(s2):
        return levenshtein2(s2, s1, deug=debug)
    
    if len(s2) == 0:
        return len(s1)
    
    if cost is None:
        cost = {}
    
    def substitution_cost(c1, c2):
        if c1 == c2:
            return 0
        return cost.get((c1, c2), 1)
    
    previous_row = range(len(s2) + 1)
    
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[i] + substitution_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))
        
        if debug:
            print(current_row[1:])
        
        previous_row = current_row
    
    return previous_row[-1]

In [10]:
s9 = '아이쿠야'
s10 = '아이쿵야'
levenshtein2(s9, s10, debug=True)

print('\n')

cost = {('쿠', '쿵'):0.1}
levenshtein2(s9, s10, cost, debug=True)

[0, 1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 1]
[2, 2, 2, 1]


[0, 1, 1, 1]
[1, 0, 1, 1]
[1, 1, 0.1, 1]
[1.1, 1.1, 1.1, 0.1]


0.1

### 초/중/종성 분리   
   
need : '쿠'와 '쿵'이 초/중/종성 중 종성만 다르니 거리를 1/3으로 정의   
   
컴퓨터는 각 글자에 대한 숫자가 정의되어 있음 (=encoding) (= 각 글자의 고유 아이디)   
Python에서 글자를 아이디로 변형하기 위해 ord 함수 사용   

In [11]:
for char in 'azAZ가힣ㄱㄴㅎㅏ':
    print('{} == {}'.format(char, ord(char)))

a == 97
z == 122
A == 65
Z == 90
가 == 44032
힣 == 55203
ㄱ == 12593
ㄴ == 12596
ㅎ == 12622
ㅏ == 12623


In [19]:
for char in ' ':
    print('{} == {}'.format(char, ord(char)))

  == 32


숫자로 표현된 글자의 고유 아이디를 글자로 변형하기 위해 chr 함수 사용

In [12]:
for idx in [97, 122, 65, 90, 44032, 55203]:
    print('{} == {}'.format(idx, chr(idx)))

97 == a
122 == z
65 == A
90 == Z
44032 == 가
55203 == 힣


완전 한글과 초/중/종성 사이에는 합성 규칙이 있음   
   
#### 초/중/종성으로 분해   
1. ord로 글자를 숫자로 변형
2. 완전 한글의 시작값 44032('가')를 뺴줌
3. 초성의 기본값(588), 중성의 기본값(28)으로 나눔
4. 몫 : 초, 중성 list의 index
5. 나머지 : 종성 list의 index
   
Composition은 반대로 진행

In [13]:
kor_begin = 44032
kor_end = 55203
chosung_base = 588
jungsung_base = 28
jaum_begin = 12593
jaum_end = 12622
moum_begin = 12623
moum_end = 12643

chosung_list = [ 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
        'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
        'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
        'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
        'ㅡ', 'ㅢ', 'ㅣ']

jongsung_list = [
    ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
        'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
        'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
        'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
              'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
              'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
              'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

def compose(chosung, jungsung, jongsung):
    char = chr(
        kor_begin + 
        chosung_base * chosung_list.index(chosung) + 
        jungsung_base * jungsung_list.index(jungsung) + 
        jongsung_list.index(jongsung)
    )
    return char

def decompose(c):
    if not character_is_korean(c):
        return None
    
    i = ord(c)
    if(jaum_begin <= i <= jaum_end):
        return (c, ' ', ' ')
    if(moum_begin <= i <= moum_end):
        return (' ', c, ' ')
    
    #deconposition rule
    i -= kor_begin
    cho = i // chosung_base
    jung = (i - cho * chosung_base) // jungsung_base
    jong = (i - cho * chosung_base - jung * jungsung_base)
    
    return (chosung_list[cho], jungsung_list[jung], jongsung_list[jong])

def character_is_korean(c):
    i = ord(c)
    return ((kor_begin <= i <= kor_end) or
           (jaum_begin <= i <= jaum_end) or
           (moum_begin <= i <= moum_end))

In [14]:
decompose('김')

('ㄱ', 'ㅣ', 'ㅁ')

In [15]:
compose('ㄱ', 'ㅣ', 'ㅁ')

'김'

### 초/중/종성 분리를 적용한 Levenshtein   
   
cost함수를 변경   
비용 0 : 두 글자가 같은 경우   
Levenshtein distance(초/중/종성 단위에서의) 계산 : 그렇지 않은 경우   
   
거리 값의 범위가 0~3이므로, 이를 3으로 나눠줌

In [16]:
def jamo_levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return jamo_levenshtein(s2, s1, debug)
    
    if len(s2) == 0:
        return len(s1)
    
    def substitution_cost(c1, c2):
        if c1 == c2:
            return 0
        return levenshtein(decompose(c1), decompose(c2))/3
    
    previous_row = range(len(s2) + 1)
    
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + substitution_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))
            
        if debug : 
            print(['%.3f'%v for v in current_row[1:]])
            
        previous_row = current_row
    
    return previous_row[-1]

In [26]:
s11 = '아이쿠야'
s12 = '아이쿵야'
jamo_levenshtein(s11, s12, debug=True)

print("\n")

s13 = '감나나다라'
s14 = '임츄듀다라'
jamo_levenshtein(s13, s14, debug=True)

['0.000', '1.000', '2.000', '3.000']
['1.000', '0.000', '1.000', '2.000']
['2.000', '1.000', '0.333', '1.333']
['3.000', '2.000', '1.333', '0.333']


['0.667', '1.667', '2.667', '3.667', '4.667']
['1.667', '1.333', '2.333', '3.000', '4.000']
['2.667', '2.333', '2.000', '2.667', '3.333']
['3.667', '3.333', '2.667', '2.000', '3.000']
['4.667', '4.333', '3.667', '3.000', '2.000']


2.0

### soynlp

In [ ]:
from soynlp.hangle import levenshtein
from soynlp.hangle import jamo_levenshtein

s1 = '아이쿠야'
s2 = '아이쿵야'

print(levenshtein(s1, s2)) # 1
print(jamo_levenshtein(s1, s2)) # 0.3333333333333333